In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # input channels = 1 for grayscale
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 128x128

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 64x64

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1))  # output: (batch, 128, 1, 1)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x.view(x.size(0), -1)  # Flatten to (batch, 128)

class SiameseClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = FeatureExtractor()

        self.classifier = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1),  # binary output
        )

    def forward(self, kernel_img, input_img):
        f1 = self.feature_extractor(kernel_img)  # shape: (B, 128)
        f2 = self.feature_extractor(input_img)

        combined = torch.cat([f1, f2], dim=1)  # shape: (B, 256)
        out = self.classifier(combined)
        return torch.sigmoid(out).squeeze(1)  # shape: (B,)

In [ ]:
def train_step(model, batch, criterion, optimizer):
    kernel_img, input_img, label = batch  # each shape: (B, 1, 256, 256)
    optimizer.zero_grad()
    output = model(kernel_img, input_img)
    loss = criterion(output, label.float())
    loss.backward()
    optimizer.step()
    return loss.item()

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import torch
import torchvision.transforms as transforms

class WaferPairDataset(Dataset):
    def __init__(self, kernel_paths, input_paths, labels, image_size=256):
        self.kernel_paths = kernel_paths
        self.input_paths = input_paths
        self.labels = labels

        # Define transforms
        self.transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=1),
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),  # Converts to (C, H, W) in [0, 1]
            transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1, 1]
        ])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        kernel_img = Image.open(self.kernel_paths[idx])
        input_img = Image.open(self.input_paths[idx])

        kernel_img = self.transform(kernel_img)
        input_img = self.transform(input_img)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)

        return kernel_img, input_img, label

In [ ]:
from torch.utils.data import DataLoader

# Example lists (replace with your actual paths and labels)
kernel_paths = ['/path/to/kernel1.png', '/path/to/kernel2.png', ...]
input_paths = ['/path/to/input1.png', '/path/to/input2.png', ...]
labels = [1, 0, ...]  # 1 = pass, 0 = fail

# Create dataset
dataset = WaferPairDataset(kernel_paths, input_paths, labels)

# Create DataLoader
train_loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

In [ ]:
import torch
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
import os

def train(model, train_loader, val_loader, num_epochs, optimizer, criterion, device, save_path='best_model.pt'):
    model = model.to(device)
    best_f1 = 0.0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        all_preds, all_labels = [], []

        for kernel, input_img, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
            kernel = kernel.to(device)
            input_img = input_img.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(kernel, input_img)  # shape: (B,)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * labels.size(0)
            all_preds.extend((outputs > 0.5).int().cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = accuracy_score(all_labels, all_preds)
        train_f1 = f1_score(all_labels, all_preds)

        # Validation
        val_loss, val_acc, val_f1 = evaluate(model, val_loader, criterion, device)

        print(f"\nEpoch {epoch+1}:")
        print(f"Train Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | F1: {train_f1:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | F1: {val_f1:.4f}")

        # Save best model
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save(model.state_dict(), save_path)
            print(f"✅ Best model saved with F1: {best_f1:.4f}")

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for kernel, input_img, labels in loader:
            kernel = kernel.to(device)
            input_img = input_img.to(device)
            labels = labels.to(device)

            outputs = model(kernel, input_img)
            loss = criterion(outputs, labels)

            total_loss += loss.item() * labels.size(0)
            all_preds.extend((outputs > 0.5).int().cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(loader.dataset)
    acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    return avg_loss, acc, f1

In [ ]:
from torch import optim
from torch.utils.data import random_split

# Assuming dataset is already created
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

# Model, loss, optimizer
model = SiameseClassifier()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train
train(model, train_loader, val_loader, num_epochs=20, optimizer=optimizer, criterion=criterion, device=device)